In [1]:
#imports
import cv2 as cv
import numpy as np

img = cv.imread('./Assets/test1.png')

img = cv.GaussianBlur(img,(5,5),0)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
mask = np.zeros((gray.shape),np.uint8)
kernel1 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(11,11))

close = cv.morphologyEx(gray,cv.MORPH_CLOSE,kernel1)
div = np.float32(gray)/(close)
res = np.uint8(cv.normalize(div,div,0,255,cv.NORM_MINMAX))
res2 = cv.cvtColor(res,cv.COLOR_GRAY2BGR)

In [2]:
# masking

thresh = cv.adaptiveThreshold(res, 255, 0, 1, 19, 2)
contour, hier = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

max_area = 0
best_cnt = None
for cnt in contour:
    area = cv.contourArea(cnt)
    if area > 1000:
        if area > max_area:
            max_area = area
            best_cnt = cnt

cv.drawContours(mask, [best_cnt], 0, 255, -1)
cv.drawContours(mask, [best_cnt], 0, 0, 2)

res = cv.bitwise_and(res, mask)
# print(res)

In [3]:
# vertical lines

kernelx = cv.getStructuringElement(cv.MORPH_RECT,(2,10))
dx = cv.Sobel(res,cv.CV_16S,1,0)
dx = cv.convertScaleAbs(dx)
cv.normalize(dx,dx,0,255,cv.NORM_MINMAX)
ret,close = cv.threshold(dx,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
close = cv.morphologyEx(close,cv.MORPH_DILATE,kernelx,iterations = 1)

contour, hier = cv.findContours(close,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

for cnt in contour:
    x,y,w,h = cv.boundingRect(cnt)
    if h/w > 5:
        cv.drawContours(close,[cnt],0,255,-1)
    else:
        cv.drawContours(close,[cnt],0,0,-1)
close = cv.morphologyEx(close,cv.MORPH_CLOSE,None,iterations = 2)


closex = close.copy()
contour, hier = cv.findContours(close,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)


# cv.drawContours(closex, contour, -1, (255,0,10), 2)
# cv.imshow("test",closex)
# cv.waitKey(0)



# for index in contour:
#     print(len(index))

cv.imshow("vertical image",close)
cv.waitKey(0)

27

In [4]:
# horizontal

kernely = cv.getStructuringElement(cv.MORPH_RECT,(10,2))
dy = cv.Sobel(res,cv.CV_16S,0,2)
dy = cv.convertScaleAbs(dy)
cv.normalize(dy,dy,0,255,cv.NORM_MINMAX)
ret,close = cv.threshold(dy,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
close = cv.morphologyEx(close,cv.MORPH_DILATE,kernely)

contour, hier = cv.findContours(close,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
for cnt in contour:
    x,y,w,h = cv.boundingRect(cnt)
    if w/h > 5:
        cv.drawContours(close,[cnt],0,255,-1)
    else:
        cv.drawContours(close,[cnt],0,0,-1)

close = cv.morphologyEx(close,cv.MORPH_DILATE,None,iterations = 2)
closey = close.copy()


cv.imshow("horizontal image",close)
cv.waitKey(0)

27

In [5]:
# result
res = cv.bitwise_and(closex,closey)
cv.imshow("final",res)
cv.waitKey(0)

27

: 